I  am working on MNIST datasets. 
This notebook is inspired from **Poonam Ligade's** kernel

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense,Dropout

In [ ]:
pd

In [ ]:
df=pd.read_csv('../input/train.csv')

In [ ]:
df.head()

In [ ]:
X_train=df.iloc[:,1:].values

In [ ]:
y_train=df.iloc[:,0].values

In [ ]:
X_test=pd.read_csv('../input/test.csv').values

In [ ]:
print(X_train.shape)
X_test.shape

In [ ]:
y_train=y_train.reshape(42000,1)

In [ ]:
X_train=X_train.reshape(X_train.shape[0],28,28)

In [ ]:
plt.imshow(X_train[0],cmap='binary')
plt.title(y_train[0])

> ***EXPANDING ONE DIMENSION FOR GRAY CODE***

In [ ]:
X_train=X_train.reshape(X_train.shape[0],28,28,1)

In [ ]:
X_test=X_test.reshape(X_test.shape[0],28,28,1)

In [ ]:
mean=X_train.mean().astype(np.float32)
std=X_train.std().astype(np.float32)

In [ ]:
mean

In [ ]:
def standardise(x):
    return (x-mean)/std

>**  One Hot Encoding**

In [ ]:
y_train.shape

In [ ]:
from keras.utils.np_utils import to_categorical
y_train=to_categorical(y_train)

In [ ]:
X_train.shape

In [ ]:
y_train.shape

-----------------------------------------------
-------------------------------------------------
***Linear Model***
-----------------------
-----------------------------------
-------------------------------------

In [ ]:
from keras.layers.core import Lambda,Flatten

    model=Sequential()
    model.add(Lambda(standardise,input_shape=(28,28,1)))

    model.add(Flatten())

    model.add(Dense(10,activation="softmax"))

In [ ]:
from keras.optimizers import RMSprop

    model.compile(optimizer=RMSprop(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
gen=ImageDataGenerator()

In [ ]:
y_train.shape

In [ ]:
from sklearn.model_selection import train_test_split
X=X_train
y=y_train
X_train,X_val,y_train,y_val=train_test_split(X_train,y_train,test_size=0.10,random_state=42)
batches=gen.flow(X_train,y_train,batch_size=64)
val_batches = gen.flow(X_val,y_val,batch_size=64)


    fitt=model.fit_generator(generator=batches,steps_per_epoch=batches.n,epochs=3,validation_data=val_batches,validation_steps=val_batches.n)

    hist_=fitt.history

    loss=hist_['loss']

    val_loss = hist_['val_loss']

    epochs= range(1,len(val_loss)+1)

    epochs

    plt.clf() #cclear plt

    plt.plot(epochs,val_loss,'bo')
    plt.plot(epochs,loss,'b+')
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.show()

    plt.clf()
    acc_values = hist_['acc']
    val_acc_values = hist_['val_acc']

    plt.plot(epochs, acc_values, 'bo')
    plt.plot(epochs, val_acc_values, 'b+')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')

    plt.show()

> **Fully connected Neural Network**
==================================

    def fcn_model():
        model=Sequential([Lambda(standardise,input_shape=(28,28,1)),
                    Flatten(),
                    Dense(512,activation='relu'),
                    Dense(10,activation='softmax')             
                  ])
        model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
        return model

    fc=fcn_model()
    fc.optimizer.lr=0.001

    fc_hist=fc.fit_generator(generator=batches,steps_per_epoch=batches.n,epochs=2,validation_data=val_batches,
                            validation_steps=val_batches.n)

## Convolutional Neural Network

In [ ]:
from keras.layers import Convolution2D , MaxPooling2D

    def cnn_model():
        model=Sequential(
        [Lambda(standardise,input_shape=(28,28,1)),
         #additional from FCN#########################################
         Convolution2D(32,(3,3),activation='relu'),
         Convolution2D(32,(3,3),activation='relu'),
         MaxPooling2D(),
         Convolution2D(64,(3,3),activation='relu'),
         Convolution2D(64,(3,3),activation='relu'),
         MaxPooling2D(),
         #############################################################
         Flatten(),
         Dense(512,activation='relu'),
         Dense(10,activation='softmax'),
        ])
        model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
        return model

    cnn_model=cnn_model()
    cnn_model.optimizer.lr=0.001


    cnn_hist=cnn_model.fit_generator(generator=batches,steps_per_epoch=batches.n,epochs=3,
                                     validation_data=val_batches,validation_steps=val_batches.n)

## Batch Normalisation

In [ ]:
from keras.layers.normalization import BatchNormalization

In [ ]:
def batch_model():
    model=Sequential([
        Lambda(standardise,input_shape=(28,28,1)),
        Convolution2D(32,(3,3),activation='relu'),
        BatchNormalization(axis=1),
        Convolution2D(32,(3,3),activation='relu'),
        MaxPooling2D(),
        BatchNormalization(axis=1),
        Convolution2D(64,(3,3),activation='relu'),
        BatchNormalization(axis=1),
        Convolution2D(64,(3,3),activation='relu'),
        MaxPooling2D(),
        Flatten(),
        BatchNormalization(axis=1),
        Dense(512,activation='relu'),
        BatchNormalization(axis=1),
        Dense(10,activation='softmax')])
    model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
    return model   

In [ ]:
complete=batch_model()
complete.optimizer.lr=0.001

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

## Submitting Predictions to Kaggle.
Using full train dataset here to train model and predict on test set.


In [ ]:
gen=ImageDataGenerator()
batches=gen.flow(X,y,batch_size=64)

In [ ]:
compl_hist = complete.fit_generator(generator=batches,steps_per_epoch=batches.n,epochs=5)

In [ ]:
predictions = complete.predict_classes(X_test,verbose=0)
submissions = pd.DataFrame({'ImageID':list(range(1,len(predictions)+1)),'Label':predictions})
submissions.to_csv('digits.csv',index=False,header=True)

In [ ]:
df=pd.read_csv('digits.csv')

In [ ]:
from IPython.display import HTML
import pandas as pd
import numpy as np
import base64

# function that takes in a dataframe and creates a text link to  
# download it (will only work for files < 2MB or so)
def create_download_link(df, title = "Download CSV file", filename = "data.csv"):  
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

# create a link to download the dataframe
create_download_link(df)